In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode("Moscow")
location




In [ ]:
location

In [ ]:
def get_cords_by_city(city:str) -> tuple:
    location = geolocator.geocode(city)
    return location[1]


In [ ]:
import requests
import config

In [ ]:
requests.get('https://api.github.com').text


In [1]:
from scheduler_grip import *

{'_id': ObjectId('63acf80b4e3114066821a242'), 'Weather': {'temp': -2.676666666666667, 'condition': 'smth', 'pressure_mm': 254.4150108, 'pressure_pa': 1009.8000000000001, 'humidity': 91.5, 'feels_like': -7.05, 'wind_dir': 'W', 'wind_speed': 9.25}, 'city': 'Moscow', 'date': '2022-12-29'}


In [2]:
get_weather_on_city('Oslo')

(59.9133301, 10.7389701)
W
C
M
U


{'merged': {'Weather': {'temp': -0.43833333333333346,
   'condition': 'smth',
   'pressure_mm': 250.27299186666667,
   'pressure_pa': 985.5333333333333,
   'humidity': 92.89999999999999,
   'feels_like': -7.06,
   'wind_dir': 'NE',
   'wind_speed': 12.196666666666667},
  'date': '2022-12-29',
  'city': 'Oslo',
  '_id': ObjectId('63ad51c61b232824235914b7')},
 'unmerged': {'Yandex': {'temp': 1,
   'condition': 'wet-snow',
   'pressure_mm': 736,
   'pressure_pa': 981,
   'humidity': 87,
   'feels_like': -14,
   'wind_dir': 'S',
   'wind_speed': 18},
  'OpenWeather': {'temp': -0.41500000000000004,
   'condition': 'Snow',
   'pressure_mm': 7.403487,
   'pressure_pa': 987,
   'humidity': 97,
   'feels_like': -3.28,
   'wind_dir': 'NNE',
   'wind_speed': 2.07},
  'VisualCrossing': {'temp': -1.9000000000000001,
   'condition': 'smth',
   'pressure_mm': 7.4079876,
   'pressure_pa': 987.6,
   'humidity': 93.7,
   'feels_like': -3.7,
   'wind_dir': 'NE',
   'wind_speed': 16.4},
  'city': 'Oslo',


In [3]:
location = get_cords_by_city('Moscow')

In [4]:
res_vis = get_vis_crossing_weather(*location)

In [5]:
res_yan = get_yandex_weather(*location)

In [6]:
res_open = get_open_weather(*location)

In [ ]:
res_open

In [ ]:
res_vis

In [ ]:
res_yan

In [ ]:
# температура | осадки | давление | влажность | ощущается как | скорость ветра 

In [ ]:
res_open['weather'][0]['main']

In [7]:
directions = [
        'N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
        'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW',
    ]
unmerged  = {
    'Yandex' : {
        'temp' : res_yan['fact']['temp'],
        'condition' : 'smth',
        'pressure_mm' : res_yan['fact']['pressure_mm'],
        'pressure_pa' : res_yan['fact']['pressure_pa'],
        'humidity' : res_yan['fact']['humidity'],
        'feels_like' :  res_yan['fact']['feels_like'],
        'wind_dir' : res_yan['fact']['wind_dir'],
        'wind_speed' : res_yan['fact']['wind_speed']
    },
    'OpenWeather' : {
        'temp' : (res_open['main']['temp_min'] + res_open['main']['temp_max'])/2,
        'condition' : 'smth',
        'pressure_mm' : res_open['main']['pressure'] *  0.007501,
        'pressure_pa' : res_open['main']['pressure'],
        'humidity' : res_open['main']['humidity'],
        'feels_like' :  res_open['main']['feels_like'],
        'wind_dir' : directions[int(round(float(res_open['wind']['deg']) / 22.5)) % 16],
        'wind_speed' : res_open['wind']['speed']
    },
    'VisualCrossing' : {
        'temp' : (res_vis['days'][0]['tempmax'] + res_vis['days'][0]['tempmin'])/2,
        'condition' : 'smth',
        'pressure_mm' : res_vis['days'][0]['pressure'] *  0.007501,
        'pressure_pa' : res_vis['days'][0]['pressure'],
        'humidity' : res_vis['days'][0]['humidity'],
        'feels_like' :  res_vis['days'][0]['feelslike'],
        'wind_dir' : directions[int(round(float(res_vis['days'][0]['winddir']) / 22.5)) % 16],
        'wind_speed' : res_vis['days'][0]['winddir'] #? бальные цифры
    }

}

In [8]:
merged = {
        "Weather": {
            "temp": (
                (res_vis["days"][0]["tempmax"] + res_vis["days"][0]["tempmin"]) / 2
                + (res_open["main"]["temp_min"] + res_open["main"]["temp_max"]) / 2
                + res_yan["fact"]["temp"]
            )
            / 3,
            "condition": "smth",
            "pressure_mm": (
                res_vis["days"][0]["pressure"] * 0.007501
                + res_open["main"]["pressure"] * 0.007501
                + res_yan["fact"]["pressure_mm"]
            )
            / 3,
            "pressure_pa": (
                res_vis["days"][0]["pressure"]
                + res_open["main"]["pressure"]
                + res_yan["fact"]["pressure_pa"]
            )
            / 3,
            "humidity": (
                res_vis["days"][0]["humidity"]
                + res_open["main"]["humidity"]
                + res_yan["fact"]["humidity"]
            )
            / 3,
            "feels_like": (
                res_vis["days"][0]["feelslike"]
                + res_open["main"]["feels_like"]
                + res_yan["fact"]["feels_like"]
            )
            / 3,
            "wind_dir": Counter(
                [
                    directions[
                        int(round(float(res_vis["days"][0]["winddir"]) / 22.5)) % 16
                    ],
                    directions[int(round(float(res_open["wind"]["deg"]) / 22.5)) % 16],
                    res_yan["fact"]["wind_dir"].upper()
                ]
            ).most_common(1)[0][0],
            "wind_speed": (
                res_vis["days"][0]["windspeed"]
                + res_open["wind"]["speed"]
                + res_yan["fact"]["wind_speed"]
            )
            / 3,
        },
    }
   


In [11]:
merged.setdefault('city', 'imre')


'imre'

In [12]:
merged

{'Weather': {'temp': -1.8066666666666666,
  'condition': 'smth',
  'pressure_mm': 254.07992723333334,
  'pressure_pa': 1008.9,
  'humidity': 93.0,
  'feels_like': -5.98,
  'wind_dir': 'W',
  'wind_speed': 8.923333333333334},
 'city': 'imre'}

In [ ]:
# get_preform_russian_cities()

In [ ]:
cities = config.cities
for item in cities.items():
    print(item[1])

In [2]:
get_preform_russian_cities_and_store_data()

S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U
S
W
C
M
U


20

In [4]:
merged_data.find_one({"city": "Moscow"})

{'_id': ObjectId('63acf80b4e3114066821a242'),
 'Weather': {'temp': -2.676666666666667,
  'condition': 'smth',
  'pressure_mm': 254.4150108,
  'pressure_pa': 1009.8000000000001,
  'humidity': 91.5,
  'feels_like': -7.05,
  'wind_dir': 'W',
  'wind_speed': 9.25},
 'city': 'Moscow',
 'date': '2022-12-29'}

2022-12-29 07:11:59,275| ERROR   | Could not establish connection from local ('127.0.0.1', 55534) to remote ('0.0.0.0', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2022-12-29 07:12:06,780| ERROR   | Could not establish connection from local ('127.0.0.1', 55534) to remote ('0.0.0.0', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2022-12-29 07:12:19,373| ERROR   | Could not establish connection from local ('127.0.0.1', 55534) to remote ('0.0.0.0', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2022-12-29 07:12:26,834| ERROR   | Could not establish connection from local ('127.0.0.1', 55534) to remote ('0.0.0.0', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2022-12-29 07:12:39,508| ERROR   | Could not establish connection from local ('127.0.0.1', 55534) to remote ('0.0.0.0', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.


In [ ]:
print(merged_data)